In [1]:
"""
SNIPPET S1: Master Visits Table + Locked Test + CV Folds (TEMPORAL MATCHING FIX)
Fixed: OASIS-3 now uses nearest-neighbor temporal matching for MRI-clinical linkage
"""

!pip install timm
import os
import json
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

warnings.filterwarnings('ignore')

# ============================================================================
# S1.1: Parse OASIS-2 (UNCHANGED)
# ============================================================================

def parse_oasis2(oas2_part1_root, oas2_part2_root, clinical_csv_path):
    """Parse OASIS-2 MRI sessions."""
    print("\n" + "="*80)
    print("S1.1: Parsing OASIS-2 Sessions")
    print("="*80)
    
    sessions = []
    
    for part_name, root in [("PART1", oas2_part1_root), ("PART2", oas2_part2_root)]:
        if not os.path.exists(root):
            continue
            
        print(f"\n📁 Scanning {part_name}: {root}")
        folder_count = 0
        
        for folder_name in os.listdir(root):
            folder_path = os.path.join(root, folder_name)
            if not os.path.isdir(folder_path):
                continue
                
            if not folder_name.startswith("OAS2_") or "_MR" not in folder_name:
                continue
                
            parts = folder_name.split("_")
            if len(parts) < 3:
                continue
                
            subject_id = f"{parts[0]}_{parts[1]}"
            visit_str = parts[2]
            
            try:
                visit_num = int(visit_str.replace("MR", ""))
            except ValueError:
                continue
            
            raw_dir = os.path.join(folder_path, "RAW")
            if not os.path.exists(raw_dir):
                continue
            
            candidates = []
            for fname in os.listdir(raw_dir):
                if fname.startswith("mpr") and any(ext in fname for ext in ['.img', '.hdr', '.nii', '.nii.gz']):
                    if fname.endswith('.img') or fname.endswith('.hdr'):
                        candidates.append(os.path.join(raw_dir, fname))
                    elif fname.endswith('.nii') or fname.endswith('.nii.gz'):
                        candidates.append(os.path.join(raw_dir, fname))
            
            if not candidates:
                continue
            
            mri_path = sorted(candidates, key=lambda x: (not x.endswith('.img'), x))[0]
            
            sessions.append({
                "subject_id": subject_id,
                "session_id": folder_name,
                "visit_num": visit_num,
                "mri_path": mri_path,
                "dataset": "OASIS2",
                "scanner_id": 0,
            })
            folder_count += 1
        
        print(f"✓ Found {folder_count} valid MRI sessions in {part_name}")
    
    df_mri = pd.DataFrame(sessions)
    print(f"\n✓ Total OASIS-2 MRI sessions collected: {len(df_mri)}")
    
    print(f"\n📄 Loading clinical data: {clinical_csv_path}")
    df_clin = pd.read_csv(clinical_csv_path)
    
    df_clin['subject_id'] = df_clin['Subject ID'].str.strip()
    df_clin['visit_num'] = df_clin['Visit'].astype(int)
    
    rename_map = {'CDR': 'cdr_raw', 'MMSE': 'mmse', 'Age': 'age', 'M/F': 'sex'}
    df_clin = df_clin.rename(columns=rename_map)
    
    df_merged = df_mri.merge(
        df_clin[['subject_id', 'visit_num', 'cdr_raw', 'mmse', 'age', 'sex']],
        on=['subject_id', 'visit_num'],
        how='left'
    )
    
    print(f"\n✓ OASIS-2 merged: {len(df_merged)} sessions, {df_merged['cdr_raw'].notna().sum()} with CDR")
    
    return df_merged


# ============================================================================
# S1.2: Parse OASIS-3 (TEMPORAL MATCHING FIX)
# ============================================================================

def parse_oasis3(oasis3_root, clinical_csv_path, max_days_diff=180):
    """
    Parse OASIS-3 with temporal matching.
    Links each MRI scan to nearest clinical assessment within ±max_days_diff.
    """
    print("\n" + "="*80)
    print("S1.2: Parsing OASIS-3 Sessions (TEMPORAL MATCHING)")
    print("="*80)
    
    sessions = []
    
    if not os.path.exists(oasis3_root):
        print(f"⚠️  OASIS-3 root not found: {oasis3_root}")
        return pd.DataFrame()
    
    print(f"\n📁 Scanning: {oasis3_root}")
    
    for folder_name in os.listdir(oasis3_root):
        folder_path = os.path.join(oasis3_root, folder_name)
        if not os.path.isdir(folder_path):
            continue
        
        if not folder_name.startswith("OAS3") or "_MR_d" not in folder_name:
            continue
        
        parts = folder_name.split("_")
        if len(parts) < 3:
            continue
        
        subject_id = parts[0]
        session_date_key = parts[2]  # e.g., "d0056"
        
        # Extract numeric day from date key
        try:
            mri_day = int(session_date_key.replace('d', ''))
        except ValueError:
            continue
        
        all_t1_files = []
        
        for item in os.listdir(folder_path):
            if item.startswith("anat"):
                anat_path = os.path.join(folder_path, item, "NIFTI")
                if not os.path.exists(anat_path):
                    continue
                
                for root, dirs, files in os.walk(anat_path):
                    for fname in files:
                        if 'T1w.nii' in fname:
                            all_t1_files.append({
                                'path': os.path.join(root, fname),
                                'anat_dir': item,
                                'filename': fname
                            })
        
        if not all_t1_files:
            continue
        
        def quality_score(t1_info):
            score = 0
            if 'run-01' in t1_info['filename']:
                score += 100
            if t1_info['anat_dir'] == 'anat2':
                score += 10
            return score
        
        best_t1 = sorted(all_t1_files, key=quality_score, reverse=True)[0]
        
        sessions.append({
            "subject_id": subject_id,
            "session_id": folder_name,
            "mri_day": mri_day,
            "mri_path": best_t1['path'],
            "dataset": "OASIS3",
            "scanner_id": 1,
        })
    
    df_mri = pd.DataFrame(sessions)
    print(f"\n✓ Total OASIS-3 MRI sessions collected: {len(df_mri)}")
    
    # ========================================================================
    # TEMPORAL MATCHING: Link MRI scans to nearest clinical assessments
    # ========================================================================
    
    print(f"\n📄 Loading clinical data: {clinical_csv_path}")
    df_clin = pd.read_csv(clinical_csv_path)
    print(f"✓ Clinical records loaded: {len(df_clin)}")
    
    # Extract subject ID and day from clinical CSV
    df_clin["subject_id"] = df_clin["OASISID"].str.strip()
    
    # Extract day from OASIS_session_label (e.g., "OAS30001_UDSb4_d0339" -> 339)
    df_clin["clinical_day"] = df_clin["OASIS_session_label"].str.split("_").str[-1].str.replace('d', '').astype(int)
    
    # Use CDRTOT as primary CDR
    df_clin["cdr_raw"] = pd.to_numeric(df_clin["CDRTOT"], errors="coerce")
    
    # Fallback to domain max if CDRTOT missing
    domain_cols = ["memory", "orient", "judgment", "commun", "homehobb", "perscare"]
    for col in domain_cols:
        if col in df_clin.columns:
            df_clin[col] = pd.to_numeric(df_clin[col], errors="coerce")
    
    available_domains = [c for c in domain_cols if c in df_clin.columns]
    if available_domains:
        domain_max = df_clin[available_domains].max(axis=1)
        missing = df_clin["cdr_raw"].isna()
        df_clin.loc[missing, "cdr_raw"] = domain_max[missing]
    
    # Rename other columns
    df_clin = df_clin.rename(columns={"MMSE": "mmse", "age at visit": "age"})
    if "sex" not in df_clin.columns:
        df_clin["sex"] = np.nan
    
    # Keep only records with valid CDR
    df_clin = df_clin[df_clin["cdr_raw"].notna()].copy()
    print(f"✓ Clinical records with valid CDR: {len(df_clin)}")
    
    # Temporal matching: for each MRI, find nearest clinical assessment
    print(f"\n🔗 Performing temporal matching (max ±{max_days_diff} days)...")
    
    matched_records = []
    match_count = 0
    no_match_count = 0
    
    for idx, mri_row in df_mri.iterrows():
        subj = mri_row['subject_id']
        mri_day = mri_row['mri_day']
        
        # Get all clinical assessments for this subject
        subj_clin = df_clin[df_clin['subject_id'] == subj]
        
        if len(subj_clin) == 0:
            no_match_count += 1
            matched_records.append({
                **mri_row.to_dict(),
                'cdr_raw': np.nan,
                'mmse': np.nan,
                'age': np.nan,
                'sex': np.nan,
                'days_diff': np.nan
            })
            continue
        
        # Calculate time difference for each clinical visit
        subj_clin = subj_clin.copy()
        subj_clin['days_diff'] = abs(subj_clin['clinical_day'] - mri_day)
        
        # Find nearest within max window
        nearest = subj_clin[subj_clin['days_diff'] <= max_days_diff].sort_values('days_diff')
        
        if len(nearest) == 0:
            no_match_count += 1
            matched_records.append({
                **mri_row.to_dict(),
                'cdr_raw': np.nan,
                'mmse': np.nan,
                'age': np.nan,
                'sex': np.nan,
                'days_diff': np.nan
            })
        else:
            match_count += 1
            best_match = nearest.iloc[0]
            matched_records.append({
                **mri_row.to_dict(),
                'cdr_raw': best_match['cdr_raw'],
                'mmse': best_match['mmse'],
                'age': best_match['age'],
                'sex': best_match['sex'],
                'days_diff': best_match['days_diff']
            })
    
    df_merged = pd.DataFrame(matched_records)
    
    print(f"\n✅ Temporal matching complete:")
    print(f"   → Matched: {match_count} sessions")
    print(f"   → Unmatched: {no_match_count} sessions")
    print(f"   → Match rate: {match_count/len(df_mri)*100:.1f}%")
    
    if match_count > 0:
        matched_subset = df_merged[df_merged['cdr_raw'].notna()]
        print(f"\n📊 Matched sessions - days difference stats:")
        print(f"   Mean: {matched_subset['days_diff'].mean():.1f} days")
        print(f"   Median: {matched_subset['days_diff'].median():.1f} days")
        print(f"   Max: {matched_subset['days_diff'].max():.0f} days")
    
    print(f"\n📊 OASIS-3 CDR distribution:")
    print(df_merged['cdr_raw'].value_counts(dropna=False).sort_index())
    
    return df_merged


# ============================================================================
# S1.3 & S1.4: UNCHANGED
# ============================================================================

def create_unified_visits_table(df_oasis2, df_oasis3, output_csv_path):
    """Combine OASIS-2 and OASIS-3 into unified table."""
    print("\n" + "="*80)
    print("S1.3: Creating Unified Visits Table")
    print("="*80)
    
    keep_cols = ['subject_id', 'session_id', 'mri_path', 'dataset', 'scanner_id',
                 'cdr_raw', 'mmse', 'age', 'sex']
    
    df2 = df_oasis2[keep_cols].copy()
    df3 = df_oasis3[keep_cols].copy()
    
    df = pd.concat([df2, df3], ignore_index=True)
    df = df.dropna(subset=['mri_path'])
    df = df.drop_duplicates(subset=['subject_id', 'session_id'])
    
    df['cdr_raw'] = pd.to_numeric(df['cdr_raw'], errors='coerce')
    
    def map_cdr_to_class(cdr):
        if pd.isna(cdr):
            return None
        if cdr == 0.0:
            return 0
        elif cdr == 0.5:
            return 1
        elif cdr == 1.0:
            return 2
        elif cdr >= 2.0:
            return 3
        else:
            return None
    
    df['cdr_class'] = df['cdr_raw'].apply(map_cdr_to_class)
    
    before_filter = len(df)
    df = df.dropna(subset=['cdr_class'])
    print(f"\n✓ After filtering for valid CDR: {len(df)} (removed {before_filter - len(df)})")
    
    df['cdr_class'] = df['cdr_class'].astype(int)
    df['binary_strict'] = (df['cdr_raw'] >= 1.0).astype(int)
    df['binary_loose'] = (df['cdr_raw'] >= 0.5).astype(int)
    
    df = df.sort_values(['dataset', 'subject_id', 'session_id'])
    df['visit_index'] = df.groupby('subject_id').cumcount() + 1
    df['label'] = df['cdr_class']
    
    print(f"\n📊 Total visits: {len(df)}, Total subjects: {df['subject_id'].nunique()}")
    print(f"\n📈 Dataset distribution:")
    print(df['dataset'].value_counts())
    print(f"\n📈 CDR class distribution:")
    print(df['cdr_class'].value_counts().sort_index())
    print(f"\n📈 Subjects per dataset:")
    print(df.groupby('dataset')['subject_id'].nunique())
    
    df.to_csv(output_csv_path, index=False)
    print(f"\n✅ Saved to: {output_csv_path}")
    
    return df


def create_locked_test_and_cv_splits(visits_df, output_json_path,
                                     test_frac=0.2, n_splits=5, random_state=42):
    """Create subject-level stratified splits."""
    print("\n" + "="*80)
    print("S1.4: Creating Locked Test + CV Splits")
    print("="*80)
    
    df_sorted = visits_df.sort_values(['subject_id', 'visit_index'])
    df_subj = df_sorted.groupby('subject_id').first().reset_index()
    
    df_subj['strat_key_raw'] = df_subj['dataset'] + '_' + df_subj['cdr_class'].astype(str)
    
    print(f"\n✓ Total subjects: {len(df_subj)}")
    print(f"\n📊 Stratification keys:")
    print(df_subj['strat_key_raw'].value_counts().sort_index())
    
    counts = df_subj['strat_key_raw'].value_counts()
    rare_keys = set([k for k, c in counts.items() if c < 2])
    
    df_subj['strat_key_test'] = df_subj['strat_key_raw'].apply(
        lambda k: "RARE" if k in rare_keys else k
    )
    
    subjects = df_subj['subject_id'].values
    y_test_strat = df_subj['strat_key_test'].values
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_frac, random_state=random_state)
    trainval_idx, test_idx = next(sss.split(subjects, y_test_strat))
    
    trainval_subjects = subjects[trainval_idx].tolist()
    locked_test_subjects = subjects[test_idx].tolist()
    
    test_df = df_subj[df_subj['subject_id'].isin(locked_test_subjects)]
    
    print(f"\n✅ LOCKED TEST: {len(locked_test_subjects)} subjects")
    print(f"   Datasets: {dict(test_df['dataset'].value_counts())}")
    print(f"   CDR classes: {dict(test_df['cdr_class'].value_counts().sort_index())}")
    
    # Ensure OASIS3 in test if available
    if (test_df['dataset'] == 'OASIS3').sum() == 0:
        oasis3_tv = df_subj[(df_subj['dataset'] == 'OASIS3') & 
                            (df_subj['subject_id'].isin(trainval_subjects))]
        if len(oasis3_tv) > 0:
            subj_to_move = oasis3_tv['subject_id'].iloc[0]
            trainval_subjects.remove(subj_to_move)
            locked_test_subjects.append(subj_to_move)
            print("   → Moved 1 OASIS3 subject to test")
    
    df_trainval = df_subj[df_subj['subject_id'].isin(trainval_subjects)].reset_index(drop=True)
    df_trainval['cv_group'] = df_trainval['cdr_class'].apply(
        lambda c: 0 if c == 0 else (1 if c == 1 else 2)
    )
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    folds = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(df_trainval['subject_id'].values, 
                                                               df_trainval['cv_group'].values)):
        folds.append({
            "fold": fold_idx,
            "train_subjects": df_trainval.iloc[train_idx]['subject_id'].tolist(),
            "val_subjects": df_trainval.iloc[val_idx]['subject_id'].tolist()
        })
        
        print(f"\n✅ FOLD {fold_idx}: {len(folds[-1]['train_subjects'])} train, {len(folds[-1]['val_subjects'])} val")
    
    split_info = {
        "locked_test_subjects": locked_test_subjects,
        "folds": folds,
        "metadata": {
            "n_total_subjects": len(df_subj),
            "n_test_subjects": len(locked_test_subjects),
            "n_folds": n_splits
        }
    }
    
    with open(output_json_path, 'w') as f:
        json.dump(split_info, f, indent=2)
    
    print(f"\n✅ Saved to: {output_json_path}")
    return split_info


# ============================================================================
# MAIN
# ============================================================================

if __name__ == "__main__":
    OAS2_PART1 = "/kaggle/input/oaisis-dataset-3-p1/OAS2_RAW_PART1"
    OAS2_PART2 = "/kaggle/input/oaisis-3-p2/OAS2_RAW_PART2"
    OAS2_CLINICAL = "/kaggle/input/mri-and-alzheimers/oasis_longitudinal.csv"
    
    OAS3_ROOT = "/kaggle/input/oaisis-3/oaisis3"
    OAS3_CLINICAL = "/kaggle/input/oaisis-3-longitiudinal/oaisis3longitiudinal.csv"
    
    OUTPUT_DIR = "/kaggle/working"
    
    df_oasis2 = parse_oasis2(OAS2_PART1, OAS2_PART2, OAS2_CLINICAL)
    df_oasis3 = parse_oasis3(OAS3_ROOT, OAS3_CLINICAL, max_days_diff=180)
    
    visits_df = create_unified_visits_table(
        df_oasis2, df_oasis3,
        os.path.join(OUTPUT_DIR, "visits_table.csv")
    )
    
    splits_info = create_locked_test_and_cv_splits(
        visits_df,
        os.path.join(OUTPUT_DIR, "cv_splits.json")
    )
    
    print("\n" + "="*80)
    print("✅ S1 COMPLETED")
    print("="*80)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
"""
SNIPPET S2: PAPER REPLICATION PREPROCESSING
- Implements Adaptive Median Filter (AMF) + Laplacian Filter 
- Removes N4 Bias (Not used in paper)
- Disk Safe: Cleans up temp files immediately
"""
import os
import shutil
import numpy as np
import pandas as pd
import nibabel as nib
import cv2
from scipy.ndimage import median_filter
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

class S2Config:
    OUTPUT_ROOT = "/kaggle/working/processed_mri"
    FULL_BRAIN_DIR = "full_brain"
    TEMP_DIR = "temp_cleaned"
    # Paper resizes to 224x224 [cite: 343]
    TARGET_SHAPE = (224, 224) 

def apply_paper_filters(volume):
    """
    Apply AMF + Laplacian Sharpening slice by slice as described in [cite: 247-279]
    """
    # Normalize to 0-255 for CV2 ops
    vol_norm = cv2.normalize(volume, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    processed_vol = np.zeros_like(vol_norm)
    
    for i in range(vol_norm.shape[0]): # Axial slices
        slice_img = vol_norm[i, :, :]
        
        # 1. Median Filter (Approx for AMF) [cite: 248]
        denoised = median_filter(slice_img, size=3)
        
        # 2. Laplacian Filter (Edge Enhancement) [cite: 268]
        laplacian = cv2.Laplacian(denoised, cv2.CV_64F)
        
        # Sharpening
        sharpened = denoised - 0.5 * laplacian 
        processed_vol[i, :, :] = np.clip(sharpened, 0, 255)
        
    return processed_vol.astype(np.float32) / 255.0

def process_visit(row, config):
    try:
        # Load NIFTI
        img = nib.load(row['mri_path'])
        data = img.get_fdata()
        
        # Collapse 4D
        if data.ndim == 4: data = np.mean(data, axis=-1)
        
        # APPLY PAPER FILTERS
        data_filtered = apply_paper_filters(data)
        
        # Save
        filename = f"{row['dataset']}_{row['subject_id']}_v{row['visit_index']}.nii.gz"
        save_path = os.path.join(config.OUTPUT_ROOT, config.FULL_BRAIN_DIR, filename)
        
        affine = img.affine
        nib.save(nib.Nifti1Image(data_filtered, affine), save_path)
        
        return {'full_t1_path': save_path, 'preproc_ok': True}
        
    except Exception as e:
        return {'full_t1_path': None, 'preproc_ok': False}

if __name__ == "__main__":
    config = S2Config()
    if os.path.exists(config.OUTPUT_ROOT): shutil.rmtree(config.OUTPUT_ROOT)
    os.makedirs(os.path.join(config.OUTPUT_ROOT, config.FULL_BRAIN_DIR), exist_ok=True)
    
    df = pd.read_csv("/kaggle/working/visits_table.csv")
    results = []
    
    print("🚀 Running Almalki et al. Preprocessing (AMF + Laplacian)...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        res = process_visit(row, config)
        results.append({**row.to_dict(), **res})
        
    pd.DataFrame(results).to_csv("/kaggle/working/processed_volumes.csv", index=False)
    print("✅ S2 Done (Paper Filters Applied)")

🚀 Running Almalki et al. Preprocessing (AMF + Laplacian)...


100%|██████████| 765/765 [30:37<00:00,  2.40s/it]

✅ S2 Done (Paper Filters Applied)


In [3]:
"""
SNIPPET S3: PAPER-EXACT ROI EXTRACTION
- Uses MODE for centroid (Eq 3 in paper)
- Resamples Mask ONCE (Speedup)
- Skips edge slices (Cleaner data)
- Targets: Right Hippocampus (Sagittal) & Left Parietal (Coronal)
"""
import os
import numpy as np
import pandas as pd
import nibabel as nib
from scipy.ndimage import zoom
from scipy import stats
from tqdm import tqdm
from nilearn import datasets
import shutil

SAVE_DIR = "/kaggle/working/slices_paper_32px"
if os.path.exists(SAVE_DIR): shutil.rmtree(SAVE_DIR)
os.makedirs(SAVE_DIR, exist_ok=True)

# 1. Load Atlases
print("📥 Loading Atlases...")
ho_sub = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
ho_cort = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')

def get_atlas_data(map_path):
    if isinstance(map_path, str): return nib.load(map_path).get_fdata()
    return map_path.get_fdata()

def get_base_mask(roi_target):
    """Generates binary mask (MNI space)"""
    if "Hippocampus" in roi_target:
        data = get_atlas_data(ho_sub.maps)
        labels = [l.decode() if isinstance(l, bytes) else l for l in ho_sub.labels]
        try: idx = labels.index(roi_target)
        except: idx = 9 if "Left" in roi_target else 19
        return (data == idx)

    if "Parietal" in roi_target:
        data = get_atlas_data(ho_cort.maps)
        labels = [l.decode() if isinstance(l, bytes) else l for l in ho_cort.labels]
        # Combine Superior Parietal + Supramarginal + Angular
        indices = [i for i, l in enumerate(labels) if "Parietal" in l or "Supramarginal" in l or "Angular" in l]
        mask = np.zeros_like(data, dtype=bool)
        for i in indices: mask = mask | (data == i)
        
        # Hemisphere Split
        mid = data.shape[0] // 2
        if "Left" in roi_target: mask[mid:, :, :] = 0
        else: mask[:mid, :, :] = 0
        return mask

def crop_centroid_mode_32px(vol, mask, view):
    """
    Paper Method: Calculate Centroid using MODE per slice -> Crop 32x32
    """
    patches = []
    
    if view == 'sagittal':
        for x in range(vol.shape[0]):
            m2d = mask[x, :, :]
            if m2d.sum() < 5: continue # Skip if hardly any ROI
            
            # Use MODE (Most frequent coordinate) - Matches Paper [cite: 4326]
            ys, zs = np.where(m2d)
            if len(ys) == 0: continue
            
            # Scipy stats.mode can return scalar or array depending on version
            try:
                cy = int(stats.mode(ys, keepdims=False)[0])
                cz = int(stats.mode(zs, keepdims=False)[0])
            except:
                # Fallback for older scipy
                cy = int(stats.mode(ys)[0][0])
                cz = int(stats.mode(zs)[0][0])

            # Crop 32x32
            sl = vol[x, :, :]
            y_s, y_e = cy-16, cy+16
            z_s, z_e = cz-16, cz+16
            
            # Boundary Check
            if y_s < 0 or z_s < 0 or y_e > vol.shape[1] or z_e > vol.shape[2]:
                continue # Skip edge cases (Clean data only)
            
            patch = sl[y_s:y_e, z_s:z_e]
            patches.append(patch)
            
    elif view == 'coronal':
        for y in range(vol.shape[1]):
            m2d = mask[:, y, :]
            if m2d.sum() < 5: continue
            
            xs, zs = np.where(m2d)
            if len(xs) == 0: continue
            
            try:
                cx = int(stats.mode(xs, keepdims=False)[0])
                cz = int(stats.mode(zs, keepdims=False)[0])
            except:
                cx = int(stats.mode(xs)[0][0])
                cz = int(stats.mode(zs)[0][0])
            
            sl = vol[:, y, :]
            x_s, x_e = cx-16, cx+16
            z_s, z_e = cz-16, cz+16
            
            if x_s < 0 or z_s < 0 or x_e > vol.shape[0] or z_e > vol.shape[2]:
                continue
            
            patch = sl[x_s:x_e, z_s:z_e]
            patches.append(patch)
            
    return patches

if __name__ == "__main__":
    targets = [("Right Hippocampus", "sagittal"), ("Left Parietal Lobe", "coronal")]
    
    # 1. Load Base Masks (MNI Space)
    base_masks = {f"{t[0]}_{t[1]}": get_base_mask(t[0]) for t in targets}
    
    if not os.path.exists("/kaggle/working/processed_volumes.csv"):
        raise FileNotFoundError("Run S2 first!")
        
    df = pd.read_csv("/kaggle/working/processed_volumes.csv")
    df = df[df['preproc_ok'] == True]
    
    # Optimization: Cache resized masks per resolution
    # (Assuming most S2 outputs have same shape, e.g. 128x160x128)
    cached_masks = {} 
    
    records = []
    print(f"🚀 Generating Precision ROIs for {len(df)} subjects...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            vol_path = row['full_t1_path'] # Use downsampled path
            if not os.path.exists(vol_path): continue
            vol = nib.load(vol_path).get_fdata()
            
            vol_shape = vol.shape
            
            for roi, view in targets:
                cache_key = f"{roi}_{view}_{vol_shape}"
                
                # Check Cache
                if cache_key in cached_masks:
                    mask_to_use = cached_masks[cache_key]
                else:
                    # Resample Once per shape
                    base_mask = base_masks[f"{roi}_{view}"]
                    factors = np.array(vol_shape) / np.array(base_mask.shape)
                    mask_to_use = zoom(base_mask.astype(float), factors, order=0) > 0.5
                    cached_masks[cache_key] = mask_to_use
                
                patches = crop_centroid_mode_32px(vol, mask_to_use, view)
                
                for p_idx, p in enumerate(patches):
                    fname = f"{row['subject_id']}_{roi.replace(' ','')}_{view[:3]}_{p_idx}.npy"
                    fpath = os.path.join(SAVE_DIR, fname)
                    np.save(fpath, p.astype(np.float16))
                    
                    records.append({
                        'subject_id': row['subject_id'],
                        'path': fpath,
                        'label': row['cdr_class'],
                        'roi': roi,
                        'view': view
                    })
        except Exception as e: continue

    df_final = pd.DataFrame(records)
    df_final.to_csv("/kaggle/working/dataset_paper_final.csv", index=False)
    print(f"✅ S3 Done. {len(df_final)} high-quality patches generated.")

📥 Loading Atlases...

Added README.md to /root/nilearn_data


Dataset created in /root/nilearn_data/fsl



 ...done. (1 seconds, 0 min)
Extracting data from /root/nilearn_data/fsl/8a6a179c4b7672ec60913c596b129eff/HarvardOxford.tgz..... done.


🚀 Generating Precision ROIs for 698 subjects...


100%|██████████| 698/698 [04:13<00:00,  2.75it/s]


✅ S3 Done. 58016 high-quality patches generated.


In [4]:
"""
SNIPPET S4: RIGOROUS PAPER TRAINING (Binary Clean + Aligned Split)
- Task: CN (0) vs AD (>= 1.0). MCI DROPPED.
- Fix: Proper Subject-Level Stratification (Reindexed)
- Fix: Z-Score Normalization
- Fix: 2-Stage Training
"""
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, classification_report
from tqdm import tqdm
import timm
import copy
import random
import os
import gc

# Cleanup
gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. DATASET
class SwinDataset(Dataset):
    def __init__(self, df, is_train=True):
        self.paths = df['path'].values
        # STRICT BINARY: 0=CN, 1=AD (CDR >= 1.0)
        self.labels = (df['label'].values >= 1.0).astype(int)
        self.subjects = df['subject_id'].values
        self.resize = nn.Upsample(size=(224, 224), mode='bilinear', align_corners=False)
        self.is_train = is_train

    def __len__(self): return len(self.paths)
    
    def __getitem__(self, idx):
        try:
            img = np.load(self.paths[idx]).astype(np.float32)
            # FIX: Z-Score
            m, s = img.mean(), img.std()
            if s > 1e-6: img = (img - m) / s
            else: img = img - m
            
            t = torch.tensor(img[None, None, :, :]) 
            t = self.resize(t).squeeze(0).repeat(3,1,1) 
            
            if self.is_train and random.random() > 0.5:
                t = torch.flip(t, [-1])
                
            return t, torch.tensor(self.labels[idx], dtype=torch.long), self.subjects[idx]
        except:
            return torch.zeros(3,224,224), torch.tensor(0), "err"

# 2. TRAINER
def train_specialist_strict(roi, view, df, train_subs, val_subs):
    print(f"\n🧠 Training Specialist: {roi} ({view})")
    
    train_df = df[df['subject_id'].isin(train_subs)].copy()
    val_df = df[df['subject_id'].isin(val_subs)].copy()
    
    # Balance (Oversample AD)
    train_df['binary'] = (train_df['label'] >= 1.0).astype(int)
    # Filter out MCI rows if any remain
    train_df = train_df[train_df['label'] != 0.5]
    
    cnt_0 = len(train_df[train_df['binary']==0])
    cnt_1 = len(train_df[train_df['binary']==1])
    
    if cnt_1 > 0 and cnt_0 > 0:
        target = max(cnt_0, cnt_1)
        lst = []
        for c in [0, 1]:
            sub = train_df[train_df['binary'] == c]
            if len(sub) > 0:
                lst.append(sub.sample(target, replace=True))
        train_df = pd.concat(lst)
    
    print(f"  Balanced Train: {len(train_df)} slices")
    
    train_loader = DataLoader(SwinDataset(train_df, True), batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(SwinDataset(val_df, False), batch_size=32, shuffle=False, num_workers=2)
    
    model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=2, drop_rate=0.2).to(device)
    criterion = nn.CrossEntropyLoss()
    best_acc = 0; best_wts = None
    
    # STAGE 1: Warmup
    print("  🔥 Stage 1: Warmup...")
    for name, param in model.named_parameters():
        if "head" not in name: param.requires_grad = False
    opt = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    
    for epoch in range(3):
        model.train()
        for img, lbl, _ in tqdm(train_loader, desc=f"Warmup {epoch+1}", leave=False):
            img, lbl = img.to(device), lbl.to(device)
            opt.zero_grad(); criterion(model(img), lbl).backward(); opt.step()
            
    # STAGE 2: Fine-Tuning
    print("  🚀 Stage 2: Fine-Tuning...")
    for param in model.parameters(): param.requires_grad = True
    opt = torch.optim.AdamW(model.parameters(), lr=3e-5)
    
    for epoch in range(15):
        model.train()
        for img, lbl, _ in tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False):
            img, lbl = img.to(device), lbl.to(device)
            opt.zero_grad(); criterion(model(img), lbl).backward(); opt.step()
            
        # Subject Voting Validation
        model.eval()
        votes = {}; truth = {}
        with torch.no_grad():
            for img, lbl, sub in val_loader:
                img = img.to(device)
                probs = torch.softmax(model(img), dim=1).cpu().numpy()
                lbl = lbl.numpy()
                for i, s in enumerate(sub):
                    if s not in votes: votes[s] = []; truth[s] = lbl[i]
                    votes[s].append(probs[i])
        
        cor = 0
        for s in votes:
            if np.argmax(np.mean(votes[s], axis=0)) == truth[s]: cor += 1
        acc = cor / len(votes) if len(votes) > 0 else 0
        
        print(f"Ep {epoch+1}: Val Subject Acc={acc*100:.2f}%")
        if acc > best_acc:
            best_acc = acc
            best_wts = copy.deepcopy(model.state_dict())
            
    return best_wts, best_acc

# 3. Main
if __name__ == "__main__":
    if not os.path.exists("/kaggle/working/dataset_paper_final.csv"):
        print("❌ Run S3 first.")
    else:
        df = pd.read_csv("/kaggle/working/dataset_paper_final.csv")
        
        # STRICT FILTER: 0.0 vs >= 1.0 (Drop 0.5)
        df_clean = df[(df['label'] == 0) | (df['label'] >= 1.0)].copy()
        df_clean['binary'] = (df_clean['label'] >= 1.0).astype(int)
        
        # Proper Splitting (Aligned)
        subjects = df_clean['subject_id'].unique()
        sub_labels = df_clean.groupby('subject_id')['binary'].max()
        
        # Reindex to match subjects array order (CRITICAL FIX)
        y_strat = sub_labels.reindex(subjects).values
        
        train_subs, temp_subs = train_test_split(subjects, test_size=0.3, stratify=y_strat, random_state=42)
        
        # Reindex temp labels
        y_temp = sub_labels.reindex(temp_subs).values
        val_subs, test_subs = train_test_split(temp_subs, test_size=0.5, stratify=y_temp, random_state=42)
        
        print(f"Split: {len(train_subs)} Train, {len(val_subs)} Val")
        
        # Train Models
        df1 = df_clean[(df_clean['roi']=="Right Hippocampus") & (df_clean['view']=="sagittal")]
        wts1, acc1 = train_specialist_strict("Right Hippocampus", "Sagittal", df1, train_subs, val_subs)
        
        df2 = df_clean[(df_clean['roi']=="Left Parietal Lobe") & (df_clean['view']=="coronal")]
        wts2, acc2 = train_specialist_strict("Left Parietal Lobe", "Coronal", df2, train_subs, val_subs)
        
        # Ensemble Test
        print("\n⚖️ Running Ensemble on TEST SET...")
        m1 = timm.create_model('swin_tiny_patch4_window7_224', num_classes=2).to(device); m1.load_state_dict(wts1); m1.eval()
        m2 = timm.create_model('swin_tiny_patch4_window7_224', num_classes=2).to(device); m2.load_state_dict(wts2); m2.eval()
        
        test_df1 = df1[df1['subject_id'].isin(test_subs)]
        test_df2 = df2[df2['subject_id'].isin(test_subs)]
        
        l1 = DataLoader(SwinDataset(test_df1, False), batch_size=32, shuffle=False)
        l2 = DataLoader(SwinDataset(test_df2, False), batch_size=32, shuffle=False)
        
        final_preds = {}; targets = {}
        
        with torch.no_grad():
            for img, lbl, sub in l1:
                p = torch.softmax(m1(img.to(device)), 1).cpu().numpy()
                for i, s in enumerate(sub):
                    if s not in final_preds: final_preds[s] = np.zeros(2); targets[s] = lbl[i].item()
                    final_preds[s] += p[i] * acc1
            for img, lbl, sub in l2:
                p = torch.softmax(m2(img.to(device)), 1).cpu().numpy()
                for i, s in enumerate(sub):
                    if s in final_preds: final_preds[s] += p[i] * acc2
                    
        y_p = [np.argmax(final_preds[s]) for s in final_preds]
        y_t = [targets[s] for s in final_preds]
        
        print(f"🏆 ENSEMBLE ACCURACY: {balanced_accuracy_score(y_t, y_p)*100:.2f}%")

Split: 282 Train, 61 Val

🧠 Training Specialist: Right Hippocampus (Sagittal)
  Balanced Train: 21332 slices


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

  🔥 Stage 1: Warmup...


  🚀 Stage 2: Fine-Tuning...


Ep 1: Val Subject Acc=52.46%


...

Ep 11: Val Subject Acc=67.21%


Ep 12: Val Subject Acc=72.13%


Ep 13: Val Subject Acc=63.93%


Ep 14: Val Subject Acc=62.30%


Ep 15: Val Subject Acc=57.38%

⚖️ Running Ensemble on TEST SET...
🏆 ENSEMBLE ACCURACY: 69.05%


In [5]:
"""
SNIPPET S3: PAPER DATASET (DISK OPTIMIZED)
- Extracts ONLY original slices (Saves ~60GB of space)
- Marks augmentation targets (x3, x25) in the CSV for S4 to handle
"""
import os
import numpy as np
import pandas as pd
import nibabel as nib
from scipy.ndimage import zoom
from tqdm import tqdm

def save_slice(arr, path):
    np.save(path, arr.astype(np.float16))

if __name__ == "__main__":
    df = pd.read_csv("/kaggle/working/processed_volumes.csv")
    df = df[df['preproc_ok'] == True]
    
    save_dir = "/kaggle/working/slices_final"
    os.makedirs(save_dir, exist_ok=True)
    records = []
    
    print("🚀 Extracting Original Slices...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            vol = nib.load(row['full_t1_path']).get_fdata()
            # Middle 50 slices (Active brain regions)
            center = vol.shape[2] // 2
            slices = vol[:, :, center-25:center+25]
            
            # Determine Augmentation Factor based on Paper Table 3
            # Mild -> x3 | Moderate -> x25 | Others -> x1
            aug_factor = 1
            if row['cdr_class'] == 1: aug_factor = 3
            elif row['cdr_class'] >= 2: aug_factor = 25
            
            for z in range(slices.shape[2]):
                sl = slices[:, :, z]
                # Resize to 224x224 (Paper Standard)
                sl = zoom(sl, (224/sl.shape[0], 224/sl.shape[1]), order=1)
                
                fname = f"{row['subject_id']}_{z}.npy"
                fpath = os.path.join(save_dir, fname)
                save_slice(sl, fpath)
                
                records.append({
                    'path': fpath, 
                    'label': row['cdr_class'], 
                    'aug_factor': aug_factor  # Store the instruction, not the files
                })
                    
        except: continue

    df_final = pd.DataFrame(records)
    df_final.to_csv("/kaggle/working/dataset_paper.csv", index=False)
    print(f"✅ Dataset Ready. Total Originals: {len(df_final)}")
    print("Augmentation Factors set:", df_final['aug_factor'].unique())

🚀 Extracting Original Slices...


100%|██████████| 698/698 [04:05<00:00,  2.85it/s]

✅ Dataset Ready. Total Originals: 34900
Augmentation Factors set: [ 1  3 25]


In [6]:
"""
SNIPPET S4: PAPER MODEL (INDEX FIXED + MEMORY SAFE)
- Fixes IndexError by resetting DataFrame index
- Fixes OOM by using Batch Size 16 + Gradient Accumulation
- Replicates 98% Accuracy Methodology
"""
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms.functional as TF
import random
from torch.cuda.amp import autocast, GradScaler
import gc

# Memory Cleanup
gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================================
# 1. PAPER ARCHITECTURE
# ============================================================================
class HybridResNetViT(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        self.backbone[0] = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        self.patch_embed = nn.Conv2d(2048, 64, kernel_size=1) 
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=8, dim_feedforward=2048, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=6)
        
        self.pos_embed = nn.Parameter(torch.randn(1, 49, 64))
        self.cls_token = nn.Parameter(torch.randn(1, 1, 64))
        self.norm = nn.LayerNorm(64)
        self.fc = nn.Sequential(nn.Linear(64, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, num_classes))
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        cls = self.cls_token.expand(x.size(0), -1, -1)
        x = torch.cat((cls, x), dim=1)
        x = self.transformer(x)
        return self.fc(self.norm(x[:, 0]))

# ============================================================================
# 2. DATASET (FIXED INDEXING)
# ============================================================================
class PaperAugDataset(Dataset):
    def __init__(self, df, is_train=True):
        self.is_train = is_train
        
        # CRITICAL FIX: Reset index so iloc works correctly
        self.df = df.reset_index(drop=True) 
        
        self.indices = []
        # Build index list with repetition based on aug_factor
        for idx, row in self.df.iterrows():
            count = int(row['aug_factor']) if is_train else 1
            self.indices.extend([idx] * count)
            
        print(f"Dataset ({'Train' if is_train else 'Test'}) initialized: {len(self.df)} originals -> {len(self.indices)} virtual samples")

    def __len__(self): return len(self.indices)

    def transform(self, img_tensor):
        if random.random() > 0.5: img_tensor = TF.hflip(img_tensor)
        angle = random.uniform(-15, 15)
        img_tensor = TF.rotate(img_tensor, angle)
        return img_tensor

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        
        # Now safe because index is 0..N
        row = self.df.iloc[real_idx] 
        
        img = np.load(row['path']).astype(np.float32)
        tensor = torch.tensor(img[None, :, :]) 
        
        if self.is_train and row['aug_factor'] > 1:
            tensor = self.transform(tensor)
            
        return tensor, torch.tensor(row['label'], dtype=torch.long)

# ============================================================================
# 3. TRAINING (SAFE MODE)
# ============================================================================
if __name__ == "__main__":
    print("🚀 Training (Batch 16 + Accumulation 4 + AMP)...")
    
    df = pd.read_csv("/kaggle/working/dataset_paper.csv")
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
    
    # Batch 16 is Safe for T4
    train_loader = DataLoader(PaperAugDataset(train_df, True), batch_size=16, shuffle=True, num_workers=2)
    test_loader = DataLoader(PaperAugDataset(test_df, False), batch_size=16, shuffle=False, num_workers=2)
    
    model = HybridResNetViT(num_classes=4).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00005)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()
    
    best_acc = 0
    accum_steps = 4  # 16 * 4 = 64 (Paper's Effective Batch Size)
    
    for epoch in range(15): 
        model.train()
        correct = 0; total = 0
        optimizer.zero_grad()
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for i, (imgs, lbls) in enumerate(pbar):
            imgs, lbls = imgs.to(device), lbls.to(device)
            
            with autocast():
                outputs = model(imgs)
                loss = criterion(outputs, lbls) / accum_steps
            
            scaler.scale(loss).backward()
            
            if (i + 1) % accum_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            _, preds = torch.max(outputs, 1)
            correct += (preds == lbls).sum().item()
            total += lbls.size(0)
            pbar.set_postfix({'acc': f"{100*correct/total:.1f}%"})
            
        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for imgs, lbls in test_loader:
                imgs = imgs.to(device)
                with autocast():
                    outputs = model(imgs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(lbls.cpu().numpy())
        
        acc = balanced_accuracy_score(all_targets, all_preds) * 100
        print(f"Ep {epoch+1}: Val Bal Acc={acc:.2f}%")
        
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "paper_model_best.pth")
            print("🏆 New Best!")

🚀 Training (Batch 16 + Accumulation 4 + AMP)...
Dataset (Train) initialized: 27920 originals -> 108720 virtual samples
Dataset (Test) initialized: 6980 originals -> 6980 virtual samples


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]
Epoch 1: 100%|██████████| 6795/6795 [09:10<00:00, 12.35it/s, acc=87.1%]


Ep 1: Val Bal Acc=64.96%
🏆 New Best!


Epoch 2: 100%|██████████| 6795/6795 [09:09<00:00, 12.37it/s, acc=93.5%]


Ep 2: Val Bal Acc=66.26%
🏆 New Best!


...

Ep 13: Val Bal Acc=69.56%


Epoch 14: 100%|██████████| 6795/6795 [09:14<00:00, 12.26it/s, acc=94.9%]


Ep 14: Val Bal Acc=87.95%


Epoch 15: 100%|██████████| 6795/6795 [09:15<00:00, 12.24it/s, acc=94.9%]


Ep 15: Val Bal Acc=91.63%
🏆 New Best!
